In [1]:
suppressMessages(library("rwwa"))

# Obs attribution

In [41]:
nsamp = 1000
rp <- 50
varnm <- "1daymax"

In [42]:
# use ensemble mean GSAT
gmst <- read.table("ts/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2024]

In [43]:
mean(gmst$gmst[(gmst$year >= 1995) & (gmst$year <= 2005)])

[1] -0.5154353

In [44]:
# use model's 2024 GMST to define factual climate        
cov_now <- gmst[gmst$year == 2024,"gmst",drop = F]
cov_cf <- rbind("pi" = cov_now - 1.3,
                "1980" = mean(gmst$gmst[(gmst$year >= 1975) & (gmst$year <= 1985)]),
                "2000" = mean(gmst$gmst[(gmst$year >= 1995) & (gmst$year <= 2005)]))

In [47]:
# now run the attribution
fl <- list.files("ts", pattern = varnm, full.names = T)

for (fnm in fl) {
    
    mnm <- substr(fnm,4,6)
    res_fnm <- paste0("res/res-obs_",varnm,"_",mnm,".csv")
    
    if(!file.exists(res_fnm)) {
    
        # load data & combine all covariates with time series
        df <- merge(load_ts(fnm, col.names = c("year", "rx1day")), gmst)
        df <- df[df$year >= 1979,]
        
        # fit the model
        mdl <- fit_ns("gev", "fixeddisp", df, "rx1day", "gmst", lower = F)

        # bootstrap results
        res <- boot_ci(mdl, cov_f = cov_now, cov_cf = cov_cf, nsamp = 1000)
        write.csv(res, res_fnm)
    }
}

# HighResMIP attribution

In [2]:
nsamp = 1000
rp <- 50
varnm <- "rx1day"

In [3]:
# use ensemble mean GSAT
gmst <- read.table("ts/gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2024]

In [4]:
# use model's 2024 GMST to define factual climate        
cov_now <- gmst[gmst$year == 2024,"gmst",drop = F]
cov_hist <- cov_now - 1.3
cov_fut <- cov_now + 0.7

In [5]:
# now run the attribution
fl <- list.files("ts", pattern = varnm, full.names = T)

for (fnm in fl) {
    
    mnm <- gsub(".dat", "", strsplit(fnm, "_")[[1]][2])
    res_fnm <- paste0("res/res-highresmip-obsgmst_",varnm,"_",mnm,".csv")
    
    if(!file.exists(res_fnm)) {
    
        # load data & combine all covariates with time series
        df <- merge(load_ts(fnm, col.names = c("year", varnm)), gmst)
        
        # fit the model
        mdl <- fit_ns("gev", "fixeddisp", df, varnm, "gmst", lower = F)

        # bootstrap results
        res <- cbind(cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
                             y_now = 2024, y_start = 1979, y_fut = 2050, nsamp = nsamp), "n_years" = nrow(df))
        write.csv(res, res_fnm)
    }
    
    # quick return level plots to check fit
    rlplot_fnm <- gsub("csv", "png", gsub("res/", "mdl-eval/rlplot_", res_fnm))
    if(!file.exists(rlplot_fnm)) {
        png(rlplot_fnm, h = 360, w = 480); par(cex.main = 1.4); {
            plot_returnlevels(mdl, cov_f = cov_now, cov_cf = cov_hist, nsamp = 100, main = mnm)
        }; dev.off()
    }
}

In [16]:
# compile results
res <- t(sapply(list.files("res", pattern = "highresmip-obsgmst_rx1day", full.names = T), read.csv, row.names = "X"))

rownames(res) <- sapply(rownames(res), function(r) gsub(".csv","",paste0(strsplit(r, "_")[[1]][c(3)], collapse = "_")))

# colnames(res) <- c(colnames(cmodel_results(mdl, rp = rp, cov_f = cov_now, cov_hist = cov_hist, cov_fut = cov_fut,
#                              y_now = 2024, y_start = 1979, y_fut = 2050, nsamp = 1)), "n_years")

write.csv(res, paste0("res-highresmip-obsgmst_",varnm,".csv"))